In [44]:
from model.LinearModel import LinearRegression
import pandas as pd
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
# load FinTech data
df = pd.read_csv('./data/FinTech.csv', encoding='utf-8')
df.head()

,Sentiment_24h,DXYUSD_Gain_Rate,World_Index_Gain_Rate,GoldUSD_Gain_Rate,Silver_Gain_Rate,DBCCommodity_Gain_Rate,DJCI_Gain_Rate,CrudeOil_Gain_Rate,SPY_Gain_Rate,VIX_Gain_Rate,TNXTbill_Gain_Rate,BTC_Price
0,0.188838,0.001341,0.003931,0.006341,0.006341,0.008765,0.019090,0.019418,0.003971,-0.000729,0.003007,117.00
1,0.169673,-0.005234,0.007311,-0.014732,-0.014732,-0.007181,-0.001202,-0.010167,0.002574,-0.013858,0.014537,103.43
2,0.131704,-0.000857,0.000000,-0.038969,-0.038969,-0.000387,-0.002442,-0.022024,-0.006590,0.043948,-0.011460,91.01
3,0.128927,0.007229,0.004812,0.009762,0.009762,0.008491,0.029350,0.032857,0.006743,-0.061464,-0.007304,118.33
4,0.157061,-0.001460,-0.012230,0.009474,0.009474,0.001896,-0.000804,0.017236,0.001427,-0.005418,0.076829,106.40


In [3]:
# Divide the data set into inputs and outputs
# df_input = df.drop(['Return'], axis=1)
# df_ouput = df[['Return']]
# df_input.shape, df_ouput.shape

df_input = df.drop(['BTC_Price'], axis=1)
df_ouput = df[['BTC_Price']]
df_input.shape, df_ouput.shape

((1912, 11), (1912, 1))

In [6]:
# 75% data is the train set, and 25% test set
X_train, X_test, Y_train, Y_test = train_test_split(df_input.values, df_ouput.values.reshape(-1,1), test_size=0.25, random_state=1)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((1434, 11), (1434, 1), (478, 11), (478, 1))

In [137]:
# # using all examples for training
# X_train, Y_train = df_input.values, df_ouput.values
# X_test = copy.deepcopy(X_train)
# Y_test = copy.deepcopy(Y_train)

In [9]:
# Normalized data
X_scaler = StandardScaler()
Y_scaler = StandardScaler()
X_scaler = X_scaler.fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

Y_scaler = Y_scaler.fit(Y_train)
Y_train = Y_scaler.transform(Y_train)
Y_test = Y_scaler.transform(Y_test)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((1434, 11), (1434, 1), (478, 11), (478, 1))

In [10]:
X_train.mean(), X_train.std()

(-9.414443500308161e-17, 0.9999999999999999)

In [11]:
X_test.mean(), X_test.std()

(0.03115018750306714, 1.0141865100438316)

In [12]:
Y_train.mean(), Y_train.std()

(-1.1396431605636196e-16, 1.0)

In [13]:
Y_test.mean(), Y_test.std()

(-0.010203548677350372, 1.0051442612138786)

In [14]:
LR_result = pd.DataFrame()

In [33]:
LR = LinearRegression()
LR.fit(X_train, Y_train, verbose=True, max_iters=10000, lr=1, print_freq=500, tol=1e-100)
LR.R2_score(X_train, Y_train)

iteration:500	 loss:0.184907
iteration:1000	 loss:0.184907
iteration:1500	 loss:0.184907
iteration:2000	 loss:0.184907
iteration:2500	 loss:0.184907
iteration:3000	 loss:0.184907
iteration:3500	 loss:0.184906
iteration:4000	 loss:0.184906
iteration:4500	 loss:0.184906
iteration:5000	 loss:0.184906
iteration:5500	 loss:0.184906
iteration:6000	 loss:0.184906
iteration:6500	 loss:0.184906
iteration:7000	 loss:0.184906
iteration:7500	 loss:0.184906
iteration:8000	 loss:0.184906
iteration:8500	 loss:0.184906
iteration:9000	 loss:0.184906
iteration:9500	 loss:0.184906
iteration:10000	 loss:0.184906


0.6301918754312268

In [27]:
LR.R2_score(X_test, Y_test)

0.6684910176940952

In [28]:
LR.MSE(X_train, Y_train)

0.3698081245687732

In [29]:
LR.MSE(X_test, Y_test)

0.33492849276793396

In [30]:
list_lambda = [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100, 300]
R2_score_train = []
R2_score_test = []
MSE_train = []
MSE_test = []
for i in list_lambda:
    LR.fit(X_train, Y_train, verbose=False, max_iters=10000, lr=i, print_freq=50)
    R2_score_train.append(LR.R2_score(X_train, Y_train))
    R2_score_test.append(LR.R2_score(X_test, Y_test))
    MSE_train.append(LR.MSE(X_train, Y_train))
    MSE_test.append(LR.MSE(X_test, Y_test))


/usr/local/Anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:38: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/home/i4ai/FinTech/model/LinearModel.py:19: RuntimeWarning: overflow encountered in square
  return 0.5 * (np.square(lables - predictions).sum() + self.lambd * np.square(self.coef_).sum()) / n_samples


In [31]:
res = pd.DataFrame()
res['lambda'] = list_lambda
res['R2_score_train'] = R2_score_train
res['R2_score_test'] = R2_score_test
res['MSE_train'] = MSE_train
res['MSE_test'] = MSE_test
res.sort_values(by=['R2_score_test'])

,lambda,R2_score_train,R2_score_test,MSE_train,MSE_test
13,300.0000,-6.176743e+292,-7.100100e+292,6.176743e+292,7.173337e+292
12,100.0000,-1.081317e+51,-1.242962e+51,1.081317e+51,1.255783e+51
0,0.0001,1.480784e-03,1.400177e-03,9.985192e-01,1.008900e+00
1,0.0003,4.431949e-03,4.396318e-03,9.955681e-01,1.005873e+00
2,0.0010,1.465264e-02,1.477537e-02,9.853474e-01,9.953872e-01
3,0.0030,4.294639e-02,4.352920e-02,9.570536e-01,9.663368e-01
4,0.0100,1.321453e-01,1.344106e-01,8.678547e-01,8.745179e-01
5,0.0300,3.193196e-01,3.266746e-01,6.806804e-01,6.802707e-01
6,0.1000,5.709811e-01,5.934947e-01,4.290189e-01,4.106984e-01
7,0.3000,6.296569e-01,6.664661e-01,3.703431e-01,3.369743e-01


In [34]:
df_import = pd.DataFrame(columns=['variable'])
df_import['variable'] = df_input.columns

In [35]:
df_coef = pd.DataFrame(columns=['variable', 'coefficient'])
df_coef['variable'] = df_input.columns
df_coef['coefficient'] = LR.coef_
df_coef['abs_coefficient'] = df_coef['coefficient'].abs()
# sort by absolute value of coefficient
pd.set_option('display.max_rows',None)
df_coef.sort_values(by=['abs_coefficient'], ascending=False)

,variable,coefficient,abs_coefficient
0,Sentiment_24h,0.791538,0.791538
8,SPY_Gain_Rate,-0.040001,0.040001
6,DJCI_Gain_Rate,0.032059,0.032059
2,World_Index_Gain_Rate,0.025413,0.025413
9,VIX_Gain_Rate,0.010414,0.010414
10,TNXTbill_Gain_Rate,-0.010344,0.010344
5,DBCCommodity_Gain_Rate,0.008497,0.008497
1,DXYUSD_Gain_Rate,-0.008275,0.008275
3,GoldUSD_Gain_Rate,0.007702,0.007702
4,Silver_Gain_Rate,0.007702,0.007702


## Other machine learning methods

In [48]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor()
clf.fit(X_train,Y_train.ravel())
clf.score(X_test,Y_test)

0.7604202337722638

In [49]:
pred = clf.predict(X_test)
MSE(Y_test, pred)

0.24205102812660773

In [50]:
pred = clf.predict(X_train)
MSE(Y_train, pred)

0.03171450304795434

In [51]:
clf.feature_importances_

array([0.81872201, 0.01761698, 0.01836733, 0.01092539, 0.01023708,
       0.0169256 , 0.02090514, 0.01867665, 0.01750007, 0.02778249,
       0.02234127])

In [52]:
rank = np.zeros(len(clf.feature_importances_))
for i, ind in enumerate(np.argsort(clf.feature_importances_)):
    rank[ind] = int(len(clf.feature_importances_) - i)
    
df_import['RandomForestRegressor'] = rank
df_import['RandomForestRegressor'] = df_import['RandomForestRegressor'].astype('int64')
df_import

,variable,RandomForestRegressor
0,Sentiment_24h,1
1,DXYUSD_Gain_Rate,7
2,World_Index_Gain_Rate,6
3,GoldUSD_Gain_Rate,10
4,Silver_Gain_Rate,11
5,DBCCommodity_Gain_Rate,9
6,DJCI_Gain_Rate,4
7,CrudeOil_Gain_Rate,5
8,SPY_Gain_Rate,8
9,VIX_Gain_Rate,2


In [55]:
from sklearn.tree import DecisionTreeRegressor
DTR = DecisionTreeRegressor()
DTR.fit(X_train, Y_train)
DTR.score(X_test,Y_test)

0.5542038393171331

In [56]:
pred = DTR.predict(X_test)
MSE(Y_test, pred)

0.45039454177282734

In [57]:
rank = np.zeros(len(clf.feature_importances_))
for i, ind in enumerate(np.argsort(DTR.feature_importances_)):
    rank[ind] = int(len(DTR.feature_importances_) - i)
    
df_import['DecisionTreeRegressor'] = rank
df_import['DecisionTreeRegressor'] = df_import['DecisionTreeRegressor'].astype('int64')
df_import

,variable,RandomForestRegressor,DecisionTreeRegressor
0,Sentiment_24h,1,1
1,DXYUSD_Gain_Rate,7,6
2,World_Index_Gain_Rate,6,7
3,GoldUSD_Gain_Rate,10,11
4,Silver_Gain_Rate,11,9
5,DBCCommodity_Gain_Rate,9,5
6,DJCI_Gain_Rate,4,3
7,CrudeOil_Gain_Rate,5,8
8,SPY_Gain_Rate,8,10
9,VIX_Gain_Rate,2,2


In [60]:
from sklearn.svm import LinearSVR
SVR = LinearSVR()
SVR.fit(X_train, Y_train.ravel())
SVR.score(X_test,Y_test)

/usr/local/Anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0.659592635505404

In [62]:
pred = SVR.predict(X_test)
MSE(Y_test, pred)

0.3439186616429999

In [63]:
rank = np.zeros(len(SVR.coef_))
for i, ind in enumerate(np.argsort(SVR.coef_)):
    rank[ind] = int(len(SVR.coef_) - i)
    
df_import['LinearSVR'] = rank
df_import['LinearSVR'] = df_import['LinearSVR'].astype('int64')
df_import

,variable,RandomForestRegressor,DecisionTreeRegressor,LinearSVR
0,Sentiment_24h,1,1,1
1,DXYUSD_Gain_Rate,7,6,10
2,World_Index_Gain_Rate,6,7,8
3,GoldUSD_Gain_Rate,10,11,7
4,Silver_Gain_Rate,11,9,6
5,DBCCommodity_Gain_Rate,9,5,2
6,DJCI_Gain_Rate,4,3,3
7,CrudeOil_Gain_Rate,5,8,9
8,SPY_Gain_Rate,8,10,5
9,VIX_Gain_Rate,2,2,4


In [71]:
df_import['sum'] = df_import['RandomForestRegressor'] + df_import['DecisionTreeRegressor'] + df_import['LinearSVR']

In [72]:
df_import.sort_values(by=['sum'])

,variable,RandomForestRegressor,DecisionTreeRegressor,LinearSVR,sum
0,Sentiment_24h,1,1,1,3
9,VIX_Gain_Rate,2,2,4,8
6,DJCI_Gain_Rate,4,3,3,10
5,DBCCommodity_Gain_Rate,9,5,2,16
10,TNXTbill_Gain_Rate,3,4,11,18
2,World_Index_Gain_Rate,6,7,8,21
7,CrudeOil_Gain_Rate,5,8,9,22
1,DXYUSD_Gain_Rate,7,6,10,23
8,SPY_Gain_Rate,8,10,5,23
4,Silver_Gain_Rate,11,9,6,26
